In [1]:
import numpy as np
import oskar
from oskar import Sky




In [4]:
#oskar_skymodel_format= r'/home/yash-op-45/Documents/simulation-ska/test_files/altaz to radec to OSKAR SkyModel format_copy.csv'
#data = np.genfromtxt(oskar_skymodel_format, delimiter=',', skip_header=1)
#print(data.shape)
skymodel = Sky()
sky=Sky.load(r"/home/yash-op-45/Documents/simulation-ska/test_files/sky.txt")


In [6]:
sky = oskar.Sky.from_array(data)
print(sky.num_sources)

RuntimeError: Too many source parameters.

In [7]:
# Specifying only RA, Dec and Stokes I (other columns default to 0).

data = np.array([[20.0, -30.0, 1],
                        [20.0, -30.5, 3],
                        [20.5, -30.5, 3]])

sky = oskar.Sky.from_array(data)

print(sky.num_sources)

3


In [8]:
sky = oskar.Sky.generate_random_power_law(100000, 1e-3, 100e-3, -2)

print(sky.num_sources)

100000


In [9]:
ra0 = 0
dec0 = 80
sky.filter_by_radius(5, 15, ra0, dec0)
print(sky.num_sources)

1429


In [ ]:
from astropy.io import fits

hdulist = fits.open(r'/home/yash-op-45/Documents/simulation-ska/test_files/vogtstar_awt.fits')

cols = hdulist[1].data[0].array

data = np.column_stack(
        (cols['RAJ2000'], cols['DEJ2000'], cols['peak_flux_wide']))

sky = oskar.Sky.from_array(data)

print(sky.num_sources)
307455